In [3]:
using CSV
using DataFrames

include("functions.jl")
using Random

In [4]:
using ScikitLearn

@sk_import svm:SVC;
@sk_import tree:DecisionTreeClassifier;
@sk_import ensemble:VotingClassifier
@sk_import neighbors: KNeighborsClassifier;


In [21]:
file_path = "dataset/simplified_Android_Malware.csv"

data = CSV.File(file_path, header=true) |> DataFrame;

# columns_to_drop = ["Flow ID", " Source IP", " Destination IP", " Timestamp"]

# select!(data, Not(columns_to_drop))
# dropmissing!(data)

In [6]:
describe(data, :all)

Row,variable,mean,std,min,q25,median,q75,max,sum,nunique,nuniqueall,nmissing,nnonmissing,first,last,eltype
,Symbol,Union…,Union…,Any,Union…,Union…,Union…,Any,Union…,Union…,Int64,Int64,Int64,Any,Any,DataType
1,Flow ID,,,1.189.213.30-10.42.0.211-443-53215-6,,,,8.8.8.8-10.42.0.211-53-9063-17,,32381,32381,0,35563,172.217.0.228-10.42.0.211-443-58310-6,10.42.0.211-10.42.0.1-26442-53-17,String
2,Source IP,,,0.0.0.0,,,,98.139.199.205,,1013,1013,0,35563,10.42.0.211,10.42.0.211,String15
3,Source Port,39136.0,17945.8,0,34535.5,43262.0,52297.0,65517,1391791790,,20464,0,35563,58310,26442,Int64
4,Destination IP,,,0.117.2.1,,,,99.199.0.77,,2435,2435,0,35563,172.217.0.228,10.42.0.1,String15
5,Destination Port,5277.97,14456.4,0.0,80.0,443.0,443.0,65165.0,1.877e8,,3398,0,35563,443.0,53.0,Float64
6,Protocol,8.33805,4.56704,0.0,6.0,6.0,6.0,17.0,296526.0,,3,0,35563,6.0,17.0,Float64
7,Timestamp,,,04/07/2017 10:05:43,,,,30/06/2017 12:59:51,,22529,22529,0,35563,15/06/2017 11:09:25,26/06/2017 12:27:44,String31
8,Flow Duration,1.07323e7,2.15277e7,2,48329.5,549353.0,1.06001e7,119977227,381673379259,,31766,0,35563,96170080,87781,Int64
9,Total Fwd Packets,7.11518,46.5218,1,1.0,2.0,5.0,3503,253037,,246,0,35563,8,1,Int64


### First approach: binary classification

In [20]:
using StatsBase

columns_to_drop = []

println("Size of dataframe before dropping columns $(size(data))")
for column in 1:size(data, 2)
    unique_values = countmap(data[:, column])

    if length(unique_values) == 1
        println("Adding column $column")
        println(unique_values)
        push!(columns_to_drop, column)
    end
    
end

select!(data, Not(columns_to_drop))

Adding column 38
Dict(0.0 => 35563)
Adding column 39
Dict(0 => 35563)
Adding column 40
Dict(0 => 35563)
Adding column 52
Dict(0.0 => 35563)
Adding column 56
Dict(0.0 => 35563)
Adding column 57
Dict(0.0 => 35563)
Adding column 63
Dict(0.0 => 35563)
Adding column 64
Dict(0.0 => 35563)
Adding column 65
Dict(0.0 => 35563)
Adding column 66
Dict(0.0 => 35563)
Adding column 67
Dict(0.0 => 35563)
Adding column 68
Dict(0.0 => 35563)


Row,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
,String,String15,Int64,String15,Float64,Float64,String31,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String31
1,172.217.0.228-10.42.0.211-443-58310-6,10.42.0.211,58310,172.217.0.228,443.0,6.0,15/06/2017 11:09:25,96170080,8,12,3734.0,1499.0,1368.0,0.0,466.75,598.266,562.0,0.0,124.917,192.683,54.414,0.207965,5.06158e6,2.19676e7,9.57763e7,13.0,9.61701e7,1.37386e7,3.61753e7,9.57763e7,151.0,9.61347e7,8.73952e6,2.88792e7,9.58134e7,13.0,0.0,256,384,0.083186,0.124779,0.0,1368.0,314.333,481.687,2.32023e5,0.0,0.0,0.0,1.0,0.0,1.0,330.05,466.75,124.917,256.0,8.0,3734.0,12.0,1499.0,1859.0,575.0,5.0,32.0,198506.0,0.0,198506.0,198506.0,9.57763e7,0.0,9.57763e7,9.57763e7,Android_Adware
2,10.42.0.211-10.42.0.1-11350-53-17,10.42.0.211,11350,10.42.0.1,53.0,17.0,15/06/2017 04:49:09,109573,1,1,34.0,199.0,34.0,34.0,34.0,0.0,199.0,199.0,199.0,0.0,2126.44,18.2527,109573.0,0.0,109573.0,109573.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32,32,9.12634,9.12634,34.0,199.0,89.0,95.2628,9075.0,0.0,0.0,0.0,0.0,0.0,1.0,133.5,34.0,199.0,32.0,1.0,34.0,1.0,199.0,-1.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Android_Adware
3,10.42.0.211-10.42.0.1-26802-53-17,10.42.0.211,26802,10.42.0.1,53.0,17.0,26/06/2017 05:03:10,47706,1,1,37.0,225.0,37.0,37.0,37.0,0.0,225.0,225.0,225.0,0.0,5491.97,41.9234,47706.0,0.0,47706.0,47706.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32,32,20.9617,20.9617,37.0,225.0,99.6667,108.542,11781.3,0.0,0.0,0.0,0.0,0.0,1.0,149.5,37.0,225.0,32.0,1.0,37.0,1.0,225.0,-1.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Android_Scareware
4,10.42.0.211-104.192.108.27-60386-443-6,10.42.0.211,60386,104.192.108.27,443.0,6.0,15/06/2017 04:11:00,20808858,8,8,343.0,5244.0,217.0,0.0,42.875,83.0309,1448.0,0.0,655.5,689.865,268.491,0.768903,1.38726e6,4.76123e6,1.85058e7,41.0,2.08089e7,2.97269e6,6.88919e6,1.85058e7,716.0,2.16747e6,309638.0,7.1848e5,1.93441e6,41.0,0.0,264,264,0.384452,0.384452,0.0,1448.0,328.647,558.774,3.12228e5,0.0,0.0,1.0,0.0,0.0,1.0,349.188,42.875,655.5,264.0,8.0,343.0,8.0,5244.0,65535.0,31.0,2.0,32.0,2.30302e6,0.0,2.30302e6,2.30302e6,1.85058e7,0.0,1.85058e7,1.85058e7,Android_Adware
5,10.42.0.211-10.42.0.1-5177-53-17,10.42.0.211,5177,10.42.0.1,53.0,17.0,15/06/2017 04:55:36,30578,1,1,37.0,53.0,37.0,37.0,37.0,0.0,53.0,53.0,53.0,0.0,2943.29,65.4065,30578.0,0.0,30578.0,30578.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,32,32.70

1

In [3]:
function transform_binary_class(output_data)
    binary_labels = output_data .!= "Benign"
    return binary_labels
end

input_data = Matrix(data[!, 1:80]);

output_data = data[!, :Label];

binary_labels = transform_binary_class(output_data)
@assert binary_labels isa BitVector
@assert input_data isa Matrix

In [4]:


Random.seed!(42)

train_indexes, test_indexes = holdOut(size(input_data, 1), 0.2)

train_input = input_data[train_indexes, :]
train_output = vec(binary_labels[train_indexes, :])

test_input = input_data[test_indexes, :]
test_output = vec(binary_labels[test_indexes, :])

@assert size(test_input, 1) == size(test_output, 1)
@assert size(train_input, 1) == size(train_output, 1)

In [7]:
# models = Dict( "SVM" => SVC(probability=true), 
#          "DT"=> DecisionTreeClassifier(max_depth=4))
# base_models =  [ name for name in keys(models)]

# for key in keys(models)
#     model = models[key]
#     fit!(model, train_input, train_output)
#     acc = score(model, test_input, test_output)
#     println("$key: $(round(acc*100, digits=2))%")
# end

In [8]:
dt = DecisionTreeClassifier(max_depth=13)
fit!(dt, train_input, train_output)
score(dt, test_input, test_output)

0.9254885421060031

In [9]:
knn = KNeighborsClassifier(13)
fit!(knn, train_input, train_output)
score(knn, test_input, test_output)

/Users/alexdopico/.julia/conda/3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.9320961619569802

In [10]:
svm = SVC(kernel = "rbf")
fit!(svm, train_input, train_output)
score(svm, test_input, test_output)

/Users/alexdopico/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9323773372697877

In [14]:
function metrics_to_latex(metrics::Dict{String, Float64})
    latex_code = """
    \\begin{tabular}{|c|c|}
    \\hline
    Metric & Value \\\\
    \\hline
    """
    for (key, value) in metrics
        latex_code *= "\\textbf{$key} & $value \\\\\n\\hline\n"
    end
    latex_code *= "\\end{tabular}"
    return latex_code
end

metrics_to_latex (generic function with 1 method)

In [7]:
kFolds = 5
crossValidationIndexes = crossvalidation(train_output, kFolds);

In [28]:
include("functions.jl")
knnParameters = Dict("modelType" => :kNN, "numNeighboors" => 0)

ks = [3, 5, 7, 10, 15, 20]
for k in ks
    knnParameters["numNeighboors"] = k
    metricsCV = (modelCrossValidation(knnParameters["modelType"], knnParameters, train_input, train_output, crossValidationIndexes))
    metrics = createAndTrainFinalModel(knnParameters["modelType"], knnParameters, train_input, train_output, test_input, test_output)
    println(k)
    for (key, value) in metrics
        if in(key, ["accuracy", "f1_score", "specificity", "recall"])
            println(key, "\t", round(value*100, digits = 2))
        else
            println(key, "\t", value)
        end
    end
end

3
errorRate	7.76
f1_score	95.95
specificity	3.53
negative_predictive_value	16.19
confusion_matrix	[6544 464; 88 17]
accuracy	92.24
recall	98.67
precision	93.38
5
errorRate	7.04
f1_score	96.34
specificity	3.12
negative_predictive_value	30.0
confusion_matrix	[6597 466; 35 15]
accuracy	92.96
recall	99.47
precision	93.4
7
errorRate	6.9
f1_score	96.42
specificity	0.62
negative_predictive_value	18.75
confusion_matrix	[6619 478; 13 3]
accuracy	93.1
recall	99.8
precision	93.26
10
errorRate	6.85
f1_score	96.45
specificity	0.21
negative_predictive_value	12.5
confusion_matrix	[6625 480; 7 1]
accuracy	93.15
recall	99.89
precision	93.24
15
errorRate	6.76
f1_score	96.5
specificity	0.0
negative_predictive_value	0.0
confusion_matrix	[6632 481; 0 0]
accuracy	93.24
recall	100.0
precision	93.24
20
errorRate	6.76
f1_score	96.5
specificity	0.0
negative_predictive_value	0.0
confusion_matrix	[6632 481; 0 0]
accuracy	93.24
recall	100.0
precision	93.24


In [38]:
include("functions.jl")
dtParameters = Dict("modelType" => :DecisionTree, "maxDepth" => 1)

depths = [3, 5, 7, 10, 15, Nothing]
for depth in depths
    knnParameters["maxDepth"] = depth
    metricsCV = (modelCrossValidation(dtParameters["modelType"], dtParameters, train_input, train_output, crossValidationIndexes))
    metrics = createAndTrainFinalModel(dtParameters["modelType"], dtParameters, train_input, train_output, test_input, test_output)
    println("-------------$depth-------------------")

    println("-------------CV-------------------")


    for (key, value) in metricsCV
        if in(key, ["accuracy", "f1_score", "specificity", "recall"])
            println(key, "\t", round(value*100, digits = 6))
        else
            println(key, "\t", value)
        end
    end

    println("--------------------------------")

    for (key, value) in metrics
        if in(key, ["accuracy", "f1_score", "specificity", "recall"])
            println(key, "\t", round(value*100, digits = 6))
        else
            println(key, "\t", value)
        end
    end
end

-------------3-------------------
-------------CV-------------------
std_recall	0.0
std_specificity	0.0
std_precision	0.0
f1_score	96.564261
specificity	0.0
std_accuracy	0.0
std_f1_score	0.0
accuracy	93.356766
recall	100.0
precision	0.9335676625659051
--------------------------------
errorRate	0.06762266273021234
f1_score	96.500546
specificity	0.0
negative_predictive_value	0.0
confusion_matrix	[6632 481; 0 0]
accuracy	93.237734
recall	100.0
precision	0.9323773372697877
-------------5-------------------
-------------CV-------------------
std_recall	0.0
std_specificity	0.0
std_precision	0.0
f1_score	96.564261
specificity	0.0
std_accuracy	0.0
std_f1_score	0.0
accuracy	93.356766
recall	100.0
precision	0.9335676625659051
--------------------------------
errorRate	0.06762266273021234
f1_score	96.500546
specificity	0.0
negative_predictive_value	0.0
confusion_matrix	[6632 481; 0 0]
accuracy	93.237734
recall	100.0
precision	0.9323773372697877
-------------7-------------------
-------------CV---

In [9]:
include("functions.jl")
svmParameters = Dict("modelType" => :SVM, "C" => 1, "kernel" => "linear", "degree" => 3, "gamma" => "scale")

svms = [
    ("rbf", 0.1),
    ("rbf", 1.0),
    ("rbf", 10.0),
    ("poly", 0.1),
    ("poly", 1.0),
    ("linear", 0.1),
    ("linear", 1.0),
    ("linear", 10.0),
]

for (kernel, C) in svms
    svmParameters["kernel"] = kernel
    svmParameters["C"] = C
    metricsCV = (modelCrossValidation(svmParameters["modelType"], svmParameters, train_input, train_output, crossValidationIndexes))
    metrics = createAndTrainFinalModel(svmParameters["modelType"], svmParameters, train_input, train_output, test_input, test_output)
    println("-------------$kernel-$C------------------")

    println("-------------CV-------------------")


    for (key, value) in metricsCV
        if in(key, ["accuracy", "f1_score", "specificity", "recall"])
            println(key, "\t", round(value*100, digits = 6))
        else
            println(key, "\t", value)
        end
    end

    println("--------------------------------")

    for (key, value) in metrics
        if in(key, ["accuracy", "f1_score", "specificity", "recall"])
            println(key, "\t", round(value*100, digits = 6))
        else
            println(key, "\t", value)
        end
    end
end

-------------rbf-0.1------------------
-------------CV-------------------
std_recall	0.0
std_specificity	0.0
std_precision	0.0
f1_score	96.564261
specificity	0.0
std_accuracy	0.0
std_f1_score	0.0
accuracy	93.356766
recall	100.0
precision	0.9335676625659051
--------------------------------
errorRate	0.06762266273021234
f1_score	96.500546
specificity	0.0
negative_predictive_value	0.0
confusion_matrix	[6632 481; 0 0]
accuracy	93.237734
recall	100.0
precision	0.9323773372697877
-------------rbf-1.0------------------
-------------CV-------------------
std_recall	0.0
std_specificity	0.0
std_precision	0.0
f1_score	96.564261
specificity	0.0
std_accuracy	0.0
std_f1_score	0.0
accuracy	93.356766
recall	100.0
precision	0.9335676625659051
--------------------------------
errorRate	0.06762266273021234
f1_score	96.500546
specificity	0.0
negative_predictive_value	0.0
confusion_matrix	[6632 481; 0 0]
accuracy	93.237734
recall	100.0
precision	0.9323773372697877
-------------rbf-10.0------------------
--

In [18]:
metrics = (modelCrossValidation(knnParameters["modelType"], knnParameters, train_input, train_output, crossValidationIndexes))

Dict{String, Float64} with 10 entries:
  "std_recall"      => 0.0
  "std_specificity" => 0.0
  "std_precision"   => 0.0
  "f1_score"        => 0.965643
  "specificity"     => 0.0
  "std_accuracy"    => 0.0
  "std_f1_score"    => 0.0
  "accuracy"        => 0.933568
  "recall"          => 1.0
  "precision"       => 0.933568

In [19]:
latex_code = metrics_to_latex(metrics)

println(latex_code)

\begin{tabular}{|c|c|}
\hline
Metric & Value \\
\hline
\textbf{std_recall} & 0.0 \\
\hline
\textbf{std_specificity} & 0.0 \\
\hline
\textbf{std_precision} & 0.0 \\
\hline
\textbf{f1_score} & 0.9656426104344664 \\
\hline
\textbf{specificity} & 0.0 \\
\hline
\textbf{std_accuracy} & 0.0 \\
\hline
\textbf{std_f1_score} & 0.0 \\
\hline
\textbf{accuracy} & 0.9335676625659051 \\
\hline
\textbf{recall} & 1.0 \\
\hline
\textbf{precision} & 0.9335676625659051 \\
\hline
\end{tabular}


In [17]:
include("functions.jl")

#topologies = [[4], [8], [2,4], [12,12]]
topologies = [[80], [40, 20]]
annParameters = Dict("modelType" => :ANN, "maxEpochs" => 1000,
    "learningRate" => 0.01, "maxEpochsVal" => 50,
    "repetitions" => 1, "validationRatio" => 0.1,
    "transferFunctions" => fill(σ, 2))

for topology in topologies
    annParameters["topology"] = topology
    # testAccuracy, testAccuracyStd, testRecall, testRecallStd = modelCrossValidation(annParameters["modelType"], annParameters, train_input, train_output, crossValidationIndexes)
    println(modelCrossValidation(annParameters["modelType"], annParameters, train_input, train_output, crossValidationIndexes))
    # println("$topology => $testAccuracy, $testAccuracyStd, $testRecall, $testRecallStd")
end


Training 1
1.0
Training 1
1.0
Training 1
1.0
Training 1
1.0
Training 1
1.0
(0.9335676625659051, 0.0, 0.06643233743409493, 0.0, 1.0, 0.0, 0.0, 0.0, 0.9335676625659051, 0.0, 0.0, 0.0, 0.9656426104344664, 0.0)
Training 1
1.0
Training 1
1.0
Training 1
1.0
Training 1
1.0
Training 1
1.0
(0.9335676625659051, 0.0, 0.06643233743409493, 0.0, 1.0, 0.0, 0.0, 0.0, 0.9335676625659051, 0.0, 0.0, 0.0, 0.9656426104344664, 0.0)


In [16]:
svmParameters = Dict("modelType" => :SVM, "degree" => 3, "gamma" => 2, "kernel" => "linear", "C" => 0.6)
testAccuracy, testAccuracyStd = modelCrossValidation(svmParameters["modelType"], svmParameters, train_input, vec(train_output), crossValidationIndexes)


In [9]:
include("functions.jl")

dtParameters = Dict("maxDepth" => 8)
knnParameters = Dict("numNeighboors" => 7)
svmParameters = Dict("kernel" => "linear", "C" => 0.5, "degree" => 3, "gamma" => 2)
Random.seed!(42)

kFolds = 2
crossValidationIndexes = crossvalidation(vec(train_output), kFolds)

trainClassEnsemble([:DecisionTree, :kNN, :SVM], [dtParameters, knnParameters, svmParameters], (train_input, train_output), crossValidationIndexes)


In [60]:
function crossvalidation_without_onehot(targets::AbstractArray{<:Any,1}, k::Int64)
    # Get the unique classes in the targets vector
    classes = unique(targets)
    # Initialize the index vector to zeros
    idx = Int.(zeros(length(targets)))
    
    # Iterate over the classes and perform stratified k-fold cross-validation
    for class in classes
        # Find the indices of the patterns that belong to the current class
        classIdx = (targets .== class)
        # Count the number of instances in the current class
        numInstances = sum(classIdx)
        # Check that there are enough instances per class to perform k-fold cross-validation
        @assert (numInstances .>= k) "There are not enough instances per class to perform a $(k)-fold cross-validation"
        # Perform stratified k-fold cross-validation on the patterns that belong to the current class
        classFolds = crossvalidation(numInstances, k)
        # Update the index vector with the indices of the folds for the current class
        idx[classIdx] .= classFolds
    end

    return idx
end

# Generate some test data
N = 20
targets = rand(["c1", "c2"], N)


# Call the crossvalidation function
crossvalidation_without_onehot(targets, 3)


20-element Vector{Int64}:
 1
 2
 2
 3
 1
 1
 3
 1
 1
 3
 2
 1
 2
 3
 3
 2
 1
 3
 2
 2

In [65]:
include("functions.jl")

crossValidationIndexes = crossvalidation_without_onehot(vec(train_output), 5)
minimum(crossValidationIndexes)

1

In [47]:
# for class in 1:size(train_output, 2)
oneHotEncoding(vec(train_output))

28450×1 BitMatrix:
 1
 0
 0
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 1
 0
 0

In [50]:
train_output isa AbstractArray{<:Any,1}

false